<a href="https://colab.research.google.com/github/KhangVGU/embedded_anti_spoofing_stm32/blob/main/nncf_test/aasist_nncf_compress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%cd /content/
!git clone https://github.com/clovaai/aasist.git

/content
fatal: destination path 'aasist' already exists and is not an empty directory.


In [17]:
%cd /content/aasist/
!pip install -r requirements.txt

/content/aasist


In [ ]:
!python ./download_dataset.py

In [2]:
# 1) Install required packages
!pip install -q torch torchvision onnx tqdm model_compression_toolkit soundfile scipy

In [14]:
!pip install numpy torch openvino fastdownload rich scikit-learn nncf soundfile scipy

In [17]:
%cd /content/aasist/

import re
import subprocess
from functools import partial
from pathlib import Path

import numpy as np
import openvino as ov
import torch
from fastdownload import FastDownload
from rich.progress import track
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
import nncf

from AASIST import Model
import soundfile as sf
from scipy.signal import resample

ROOT = Path("/content/aasist/")
AUDIO_DIR = ROOT / "/content/aasist/LA/ASVspoof2019_LA_dev/flac"
BATCH_SIZE = 1                          # batch size for calibration & validation
SUBSET_SIZE = 300 // BATCH_SIZE         # NNCF default ~300 samples
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/content/aasist


In [18]:
# 1) Audio dataset

class AudioFolderDataset(Dataset):
    """Loads all .flac files under AUDIO_DIR."""
    def __init__(self, root_dir: Path, target_sr=16000):
        self.files = sorted(root_dir.glob("*.flac"))
        self.sr = target_sr

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        wav, sr = sf.read(str(self.files[idx]))
        if sr != self.sr:
            n = int(len(wav) * self.sr / sr)
            wav = resample(wav, n)
        wav = torch.from_numpy(wav.astype(np.float32))
        # if you have labels in filename or folder structure,
        # return them here; for now we just return dummy zero.
        return wav, 0

# transform_fn for NNCF calibration: strip label, send to device
def transform_fn(data_item, device):
    wav, _ = data_item
    # ensure shape (1, T)
    return wav.unsqueeze(0).to(device)

In [21]:
# 2) Load model

# load your training config to get d_args
import json
cfg = json.load(open(ROOT/"config/AASIST.conf"))
d_args = cfg["model_config"]
# sanity check
if not isinstance(d_args["filts"][0], int):
    raise ValueError("d_args['filts'][0] must be int")

# instantiate & load checkpoint
model = Model(d_args).to(DEVICE)
ckpt = torch.load(ROOT/"models/weights/AASIST.pth", map_location=DEVICE)
sd = ckpt.get("state_dict", ckpt)
res = model.load_state_dict(sd, strict=False)
if res.unexpected_keys: print("Unexpected:", res.unexpected_keys)
if res.missing_keys:    print("Missing   :", res.missing_keys)
model.eval()
torch.set_grad_enabled(False)

In [22]:
# 3) Prepare calibration & validation loaders

dataset = AudioFolderDataset(AUDIO_DIR, target_sr=16000)
calib_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(dataset, batch_size=1,      shuffle=False)

# NNCF calibration dataset
calib_ds = nncf.Dataset(calib_loader, partial(transform_fn, device=DEVICE))

In [23]:
# 4) Quantize with NNCF

torch_quant = nncf.quantize(
    model.cpu(),              # quantize on CPU
    calib_ds,
    subset_size=SUBSET_SIZE
).to(DEVICE)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


Output()

INFO:nncf:Compiling and loading torch extension: quantized_functions_cpu...
INFO:nncf:Finished loading torch extension: quantized_functions_cpu


Output()

In [26]:
# 5) Convert both FP32 & INT8 to OpenVINO IR

# derive dummy_input from first sample length
first_wav, _ = dataset[0]
T = first_wav.shape[-1]
dummy = torch.randn(1, T)

ov_fp32 = ov.convert_model(model.cpu(), example_input=dummy)
ov_int8 = ov.convert_model(torch_quant.cpu(), example_input=dummy)

fp32_xml = ROOT/"aasist_fp32.xml"
ov.save_model(ov_fp32, fp32_xml, compress_to_fp16=False)
int8_xml = ROOT/"aasist_int8.xml"
ov.save_model(ov_int8, int8_xml)

RuntimeError: Couldn't get TorchScript module by tracing.
Exception:
Given input size: (70x1x23360). Calculated output size: (70x0x7786). Output size is too small
Please check correctness of provided 'example_input'. Sometimes models can be converted in scripted mode, please try running conversion without 'example_input'.
 You can also provide TorchScript module that you obtained yourself, please refer to PyTorch documentation: https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html.